In [2]:
%matplotlib inline
from ultralytics import YOLO
from pathlib import Path
import cv2
import shutil
from generate_html import generate_comparison_table, generate_comparison_html, generate_comparison_html_1
import random

# 复制图片
def copy_images_to_new_folder(target_folder, test_images):
    # 创建目标文件夹
    if target_folder.exists():
        shutil.rmtree(target_folder)
        print(f"Folder already exists: {target_folder}")

    target_folder.mkdir(parents=True, exist_ok=True)
    print(f"Created folder: {target_folder}")

    # 遍历每张图片并复制到目标文件夹
    for image_path in test_images:
        source_path = Path(image_path)
        target_path = target_folder / source_path.name

        # 复制图片
        try:
            shutil.copy(source_path, target_path)
            print(f"Copied {source_path} to {target_path}")
        except Exception as e:
            print(f"Failed to copy {source_path}: {e}")

# 拼接测试图片路径
# type=1 随机
def generate_test_image_paths(type=0):
    # 替换成要测试的图片文件夹
    path = '../datasets/YOLO_format_widerface/test/images/'
    images = []
    if type == 1:
        # 随机选择图像
        images_dir = Path("../datasets/YOLO_format_widerface/test/images")
        all_images = list(images_dir.glob("*.jpg"))  # 假设图像是 JPG 格式，若为 PNG，请改为 "*.png"
        # 选择某个类别的图片
        # all_images = [image for image in images_dir.glob("*") if image.name.startswith("56_")]
        random_images = random.sample(all_images, 20)
        images = [image.name for image in random_images]
    else:
        images = [
            '56_Voter_peoplevoting_56_342.jpg', # good
            '14_Traffic_Traffic_14_766.jpg',  # good
            '32_Worker_Laborer_Worker_Laborer_32_423.jpg', # good
            '52_Photographers_photographertakingphoto_52_131.jpg', # good
            '3_Riot_Riot_3_561.jpg',
            '42_Car_Racing_Car_Racing_42_175.jpg',
            '42_Car_Racing_Car_Racing_42_817.jpg',
            '42_Car_Racing_Nascar_42_665.jpg',
            '56_Voter_peoplevoting_56_256.jpg',
            '56_Voter_peoplevoting_56_258.jpg',
            '56_Voter_peoplevoting_56_677.jpg',
            '0_Parade_marchingband_1_196.jpg'
            # '42_Car_Racing_Nascar_42_700.jpg',
            # '42_Car_Racing_Nascar_42_657.jpg',
            # '42_Car_Racing_Nascar_42_333.jpg',
        ]
    return [path + image for image in images]

# 创建文件夹
def create_folder(models, input_sizes, base_path):
    # 遍历每个模型和每个输入尺寸，创建文件夹
    for model in models:
        for input_size in input_sizes:
            dir_path = base_path / f"model_{model}" / input_size
            if dir_path.exists():
                print(f"Directory already exists: {dir_path}")
                shutil.rmtree(dir_path)
                dir_path.mkdir(parents=True, exist_ok=True)
                print(f"Created directory: {dir_path}")

            dir_path.mkdir(parents=True, exist_ok=True)
            print(f"Created directory: {dir_path}")


def inference(model, input_img_size, cnf, folder_path):
    # 对每张图像进行推理并显示
    for img_path in test_set_images:
        # Read the image
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Failed to load image: {img_path}")
            continue
        # 使用模型进行推理
        results = model.predict(source=img, save=False, imgsz=input_img_size,conf=cnf)

        # 获取推理后的图像，添加检测框
        # result_img = results[0].plot()  # YOLOv8 API 中可直接使用 plot() 获取带标注的图像
        boxes = results[0].boxes
        # Apply Gaussian blur to each detected region
        for box in boxes:
            # Extract the bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Ensure coordinates are within image bounds
            x1, y1 = max(x1, 0), max(y1, 0)
            x2, y2 = min(x2, img.shape[1]), min(y2, img.shape[0])

            # Extract the region of interest (ROI)
            roi = img[y1:y2, x1:x2]

            # Apply Gaussian blur to the ROI
            blurred_roi = cv2.GaussianBlur(roi, (51, 51), 0)

            # Place the blurred ROI back into the original image
            img[y1:y2, x1:x2] = blurred_roi

            # 繪製邊框
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # 顯示置信度
            label = f"{box.conf[0]:.2f}"
            label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            label_y = y1 - 10 if y1 - 10 > 10 else y1 + 10
            cv2.putText(img, label, (x1, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        image_name = Path(img_path).name
        output_path = folder_path / image_name
        print(f"output_path: {output_path}")
        cv2.imwrite(str(output_path), img)
        print(f"Processed image saved to: {output_path}")

def generate_html(type, models, base_path, img_path, input_sizes):
    if type == 1:
         # 调用函数生成每个模型的 HTML
        for model in models:
            generate_comparison_table(model,base_path, img_path, input_sizes)
    elif type == 2:
        generate_comparison_html(models, base_path, img_path, input_sizes)
    elif type == 3:
        generate_comparison_html_1(models, base_path, img_path, input_sizes)
# 模型名称和输入尺寸
models = [
    "416",
    "512",
    "640",
    "focal_0.1",
    # "SPDConv"
]
input_sizes = [192, 256, 512]
base_path = Path("./results")
cnf = 0.5  # 置信度阈值
img_path = Path("./results/img")

# 拼接测试图片路径
test_set_images = generate_test_image_paths()
# 从 dataset 中复制 图片到指定文件夹
copy_images_to_new_folder(img_path, test_set_images)
# 创建结果文件夹
create_folder(models, ["192", "256", "512"], base_path)

# 主逻辑
def main():
    # 遍历每个模型和每个输入尺寸
    for model_name in models:
        model_path = f"models/{model_name}/weights/best.pt"
        model = YOLO(model_path)  # 加载模型

        for input_size in input_sizes:
            folder_path = base_path / f"model_{model_name}" / str(input_size)
            print(f"Running inference for model {model_name} with input size {input_size}")
            inference(model, input_size, cnf, folder_path)

    generate_html(3, models, base_path, img_path, input_sizes)

main()

Folder already exists: results/img
Created folder: results/img
Copied ../datasets/YOLO_format_widerface/test/images/56_Voter_peoplevoting_56_342.jpg to results/img/56_Voter_peoplevoting_56_342.jpg
Copied ../datasets/YOLO_format_widerface/test/images/14_Traffic_Traffic_14_766.jpg to results/img/14_Traffic_Traffic_14_766.jpg
Copied ../datasets/YOLO_format_widerface/test/images/32_Worker_Laborer_Worker_Laborer_32_423.jpg to results/img/32_Worker_Laborer_Worker_Laborer_32_423.jpg
Copied ../datasets/YOLO_format_widerface/test/images/52_Photographers_photographertakingphoto_52_131.jpg to results/img/52_Photographers_photographertakingphoto_52_131.jpg
Copied ../datasets/YOLO_format_widerface/test/images/3_Riot_Riot_3_561.jpg to results/img/3_Riot_Riot_3_561.jpg
Copied ../datasets/YOLO_format_widerface/test/images/42_Car_Racing_Car_Racing_42_175.jpg to results/img/42_Car_Racing_Car_Racing_42_175.jpg
Copied ../datasets/YOLO_format_widerface/test/images/42_Car_Racing_Car_Racing_42_817.jpg to res